In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from joblib import Parallel, delayed
from ipywidgets import IntProgress
from matplotlib.collections import LineCollection
import matplotlib
import matplotlib.collections as mcoll
import random

# Local imports
from searchdir import searchdir

matplotlib.use('Agg')

# Constants for the Notebook
path_to_dir = "/home/nikolas/Documents/Research/PMT_Testing/Map"
paths = searchdir(path_to_dir, file_extension=".h5")

# Scale Factors
mppc_DIGITIZER_FULL_SCALE_RANGE = 2.5 # Vpp
mppc_DIGITIZER_RESOLUTION = 12 # bits
digiCounts = 2.**mppc_DIGITIZER_RESOLUTION
verticalScaleFactor = 1.e3*mppc_DIGITIZER_FULL_SCALE_RANGE/digiCounts; # mV/bank
mppc_DIGITIZER_SAMPLE_RATE = 3200000000 # S/s
horizontalScaleFactor = 1.e9/mppc_DIGITIZER_SAMPLE_RATE #ns/sample

print(paths)

Run_IDs = [s.split("/")[-1].split(".")[0][-3:] for s in paths]
RunPathMap = {Run: path for Run, path in zip(Run_IDs, paths)}
PathRunMap = {path: Run for Run, path in zip(Run_IDs, paths)}
Run_IDs.sort()
RunLocMap = {Run: i for i, Run in enumerate(Run_IDs)}

print(RunLocMap)

# print(Run)

### Functions used in this notebook

In [173]:
def filterSignal(path:str, channels:list[str], highpass=600, lowpass=2000, filterChannel=None, timeThreshold=80, ADCThreshold=800, peakMinTimeThreshold=20):
    """Filters waveforms based on highpass and lowpass filter. Can be used to drop oversaturated datat (lowpass) and to drop outliers (highpass).

    Args:
        path (str): Path to the HDF file containing the waveforms.
        channels (list[str]): A list of channels in the HDF keys (["Ch0", "Ch1", "Ch2", "Ch3"]).
        highpass (int, optional): Highpass filter. Defaults to 600.
        lowpass (int, optional): Lowpass filter. Defaults to 2000.

    Returns:
        _type_: _description_
    """
    print(f"Cleaning waveforms in: {path}")
    DATA = {ch: None for ch in channels}
    for channel in channels:
        df = pd.read_hdf(path, key=channel)
        
        # Check if filters are different for each channel
        hp = highpass
        lp = lowpass
        if type(highpass) is dict:
            if filterChannel is None:
                filterChannel = channel
            hp = highpass[filterChannel]
        if type(lowpass) is dict:
            if filterChannel is None:
                filterChannel = channel
            lp = lowpass[filterChannel]
            
        if highpass is None and lowpass is None:
            DATA[channel] = df
            continue
        elif highpass is None:
            mask = (df.min(axis=1) <= lp)
        elif lowpass is None:
            mask = (df.max(axis=1) >= hp)
        else:
            print(f"Applying filters: [{hp}:{lp}] on {filterChannel}")
            mask = ((df <= lp) & (df >= hp)).all(axis=1)

        # Apply the mask to filter rows
        filtered_df = df[mask]
        
        # Drop waveforms with peak beyond 40 ns
        j_max = filtered_df.idxmin(axis=1)
        
        # Convert the index to numeric
        j_max_numeric = pd.to_numeric(j_max, errors='coerce') * horizontalScaleFactor
        
        # Filter rows based on the peak time (j_max_numeric) being <= 40 ns
        filtered_df = filtered_df[(j_max_numeric >= peakMinTimeThreshold)]
        
        # Drop waveforms with any value greater than the threshold beyond 50 ns
        # Calculate the index position for 50 ns
        time_50ns = int(timeThreshold / horizontalScaleFactor)
                
        # Apply filter: remove rows where any value after 50 ns exceeds threshold
        filtered_df = filtered_df[filtered_df.iloc[:, time_50ns:].min(axis=1) >= ADCThreshold]
        
        # Save data
        DATA[channel] = filtered_df
        
        dropped = df.shape[0] - filtered_df.shape[0]
        print(f"Dropped {dropped} waveforms from: {filterChannel}")
        
    return DATA


def averageWaveforms(Traw:pd.DataFrame, bin_size:int=100):
    """Average the waveforms from the raw dataframe into bins. 

    Args:
        Traw (pd.DataFrame): Dataframe containing waveforms
        bin_size (int, optional): Width of bin used for averaging. Defaults to 100. 
                                  None will average all waveforms to a single bin.

    Returns:
        Tav (pd.DataFrame): Average dataframe with bins of size bin_size.
    """
    # Group rows into bins of size `bin_size` and calculate the mean for each bin
    if bin_size != None:
        Tav = Traw.groupby(Traw.index // bin_size).mean()
    elif bin_size == 1:
        Tav = Traw
    else:
        Tav = Traw.mean()
        
    # Reset the index for clarity
    Tav.reset_index(drop=True, inplace=True)
    
    return Tav


def baselineWaveforms(Tin:pd.DataFrame, mode="poly", regions=[(0, 50), (300, 512)]):
    """
    Perform baseline correction for waveforms in a DataFrame by fitting and subtracting linear trends.

    Args:
        Tin (pd.DataFrame): Input DataFrame where each row represents a waveform, and columns represent time steps.
        regions (list of tuples): List of regions (start, end) used to estimate the baseline. 
                                  Defaults to [(0, 50), (300, 512)].

    Returns:
        tuple: A tuple containing:
            - out (pd.DataFrame): The baseline-corrected waveforms.
            - pd.DataFrame: The fitted baseline values for each waveform.
    """
    # Baseline Region: Combine specified regions
    baselineRaw = pd.concat([Tin.iloc[:, regions[i][0]:regions[i][1]] for i in range(len(regions))], axis=1)

    if mode == "mean":
        # Calculate the mean baseline for each row
        meanBase = baselineRaw.mean(axis=1)
        # Subtract mean baseline from the original data
        result = Tin.sub(meanBase, axis=0)
        return result, meanBase

    elif mode == "polyfit":
        # Concatenate the indices for the specified regions
        X_fit = np.concatenate([np.arange(regions[i][0], regions[i][1]) for i in range(len(regions))])
        # Get the raw NumPy array for baseline fitting
        Y = baselineRaw.to_numpy()

        # Initialize an array to store polynomial coefficients
        coeffs = []
        
        # Compute polynomial fits
        coeffs = Parallel(n_jobs=-1)(delayed(np.polyfit)(X_fit, y, deg=0) for y in Y)

        # Convert the list of coefficients to a NumPy array
        coeffs = np.array(coeffs)

        # Evaluate the polynomial fits using np.polyval
        X_val = np.arange(Tin.shape[1])
        fitted_lines = np.array([np.polyval(p, X_val) for p in coeffs])

        # Convert fitted lines back to a Dask DataFrame
        fitted_lines_df = pd.DataFrame(fitted_lines, index=Tin.index, columns=Tin.columns)

        # Subtract polynomial fit from the original data
        result = Tin.sub(fitted_lines_df)

        return result, pd.DataFrame(fitted_lines)

    else:
        raise ValueError("Invalid mode. Use 'mean' or 'polyfit'.")


def displayWaveforms(T: dict, xscale=horizontalScaleFactor, title="", vlines=None, save=True, showplot=False, savepath="Figures/", rowwise=True, multi=False):
    """
    Display multiple waveforms stored in a dictionary of DataFrames as subplots.

    Args:
        T (dict): Dictionary where keys are channel identifiers and values are DataFrames containing waveforms.
        xscale (float): Scaling factor for the x-axis (time). Default is 1.0.
        title (str): Title for the overall figure.
        vlines (dict): Dictionary with channel names as keys and lists of vertical line positions as values.

    Returns:
        None: Displays the waveforms as a series of subplots.
    """
    if not showplot:
        plt.ioff()
    
    channels = list(T.keys())
    nrows = int(len(channels) / 2) if len(channels) % 2 == 0 else int(len(channels) // 2 + 1)
    ncols = 2
    figsize = (16,10)
    
    if not rowwise:
        ncols = nrows
        nrows = 2
        figsize = (10, 16)
        
    if not multi:
        ncols = 1
        nrows = 1
    
    # Create figure and axes for subplots
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    if nrows > 1 or ncols > 1:
        axes = axes.ravel()
    else:
        axes = [axes]
    cmap = plt.get_cmap('tab10')  # Use colormap for line colors
    
    # Determine the device for PyTorch
    # device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    for i, ax in enumerate(axes):
        if i < len(channels):
            channel = channels[i]
            df = T[channel].T

            # Extract X and Y data
            X = df.index.values * xscale  # Shape: (num_points,)
            Y = df.values  # Shape: (num_points, num_waveforms)

            print(f"Plotting ID {channel} with {Y.shape[1]} waveform(s)...")

            # Prepare segments and colors
            segments = []
            colors = []

            for idx, waveform in enumerate(Y.T):  # Iterate over each waveform (column of Y)
                points = np.stack((X, waveform), axis=1)  # Shape: (num_points, 2)
                segments.append(points)

                # Assign color based on the waveform index
                c  = random.random()
                colors.append(cmap(c))

            # Create a LineCollection from segments and colors
            lc = mcoll.LineCollection(segments, linewidths=0.5, colors=colors, alpha=0.6, rasterized=True)
            ax.add_collection(lc)
            ax.autoscale()  # Automatically adjust limits to fit data

            ax.set_title(f"Run {channel}")
            ax.set_xlabel("Time (ns)")
            ax.set_ylabel("Amplitude")

            if vlines is not None and channel in vlines:
                for v in vlines[channel]:
                    ax.axvline(x=v * xscale, linestyle='--', color='black', alpha=0.6, rasterized=True, linewidth=0.7)
        else:
            ax.axis("off")  # Turn off unused subplots if there are extra axes
    
    fig.suptitle(title)
    plt.tight_layout()
    
    if save:
        fname = savepath+title.replace(" ", "_")+".png"
        print(f"Saving figure to {fname}...")
        plt.savefig(fname, dpi=100)  # JPEG saves faster
        print("Done")
        plt.close()
    
    if showplot:
        print("Rendering figure")
        plt.show()
    
    
def analyzeWaveforms(Tin:pd.DataFrame, j_A:int=50, j_B:int=100):
    """
    Analyze waveforms in a DataFrame and return results as a DataFrame.
    
    Args:
        Tin (pd.DataFrame): Each row is a waveform; each column is a time step.
        j_A (int): Number of time steps before the peak to include in integration.
        j_B (int): Number of time steps after the peak to include in integration.
    
    Returns:
        data (pd.DataFrame): DataFrame containing charge, amplitude, and timing.
    """
    # Index of maximum value in pulse (for each row)
    j_max = Tin.idxmax(axis=1)

    # Timing (location of the pulse peak)
    timing = j_max

    # Nominal Baseline value (average of first 50 columns for each row)
    baseline = Tin.iloc[:, 0:50].mean(axis=1)

    # Amplitude of the pulse -- Define as height between maximum and the baseline
    amplitude = Tin.max(axis=1) - baseline

    # Create a mask for charge integration
    A = (j_max - j_A).clip(lower=0)  # Ensure indices are not below 0
    B = (j_max + j_B).clip(upper=Tin.shape[1])  # Ensure indices do not exceed max columns

    # Vectorized integration
    charge = [
        (Tin.iloc[i, A[i]:B[i]] - baseline.iloc[i]).sum() for i in range(len(Tin))
    ]

    # Combine results into a DataFrame
    data = pd.DataFrame({
        "charge": charge,
        "amplitude": amplitude,
        "timing": timing
    })

    return data, (A.mean(), B.mean())

### Process Calibration Data

- Calibration data is saved in individual files. The data is extracted and filtered using the `filterSignal()` function.
- Calibration data is then averaged using the `averageWaveform()` function into bins of `1000` samples.
- Then, a baseline reduction is performed on the waveforms using the `baseline()` function. This subtracts out a 1 degree polynomial fit from the baseline.
- Finally, the signal is flipped so it the information from the waveform can be extracted.
- Visualisations of the the data is produced at each step (when applicable)

In [ ]:
progress = IntProgress(min=0, max=(len(paths))*5)
display(progress)

ID = Run_IDs

# Load and filter signals
Traw_cal = {i: None for i in ID}
T = {i: None for i in ID}  # For the plot

highpass = {"Ch0": 0}
for path in paths:
    Traw_cal[PathRunMap[path]] = filterSignal(path, ["Ch0"], highpass=0, lowpass=5000, timeThreshold=0, ADCThreshold=0, peakMinTimeThreshold=0)["Ch0"]  # Filtered
    progress.value += 5
    
# Average waveforms
Tav_cal = {ch: averageWaveforms(Traw_cal[ch], bin_size = 1) for ch in ID}

progress.value += 5

# Baseline reduction of waveforms
Tbase_cal = {ch: None for ch in ID}
fitted_baseline_cal = {ch: None for ch in ID}
for ch in ID:
    Tbase_cal[ch], fitted_baseline_cal[ch] = baselineWaveforms(Tav_cal[ch], mode="polyfit")#, regions=[(0, 10)])#, (300, 512)])#, regions=[(0, 50)])
    progress.value += 5

# Update progress bar
progress.value += len(ID) * 5

# Flip the waveforms for the data extraction from the waveforms
Tflip_cal = {ch: -1*Tbase_cal[ch] for ch in ID}

# Save data as figures
save=False
if save:
    XY = {"Ch0": Traw_cal["Ch0"].iloc[0:1000]}
    XY2 = {"Ch0": fitted_baseline_cal["Ch0"].iloc[0:1000]}
    # Display Filtered waveforms
    # displayWaveforms(XY, title=Run + " Raw Sensitivity", save=True)

    # Display averaged waveforms
    displayWaveforms(Tav_cal, title="Processed Averaged Waveforms", save=True)

    # Display the baseline fit that was subtracted out
    # displayWaveforms(XY2, title= Run+" Baselined Sensitivity", save=True)

    # Display baseline reduced waveforms
    displayWaveforms({"Ch0": fitted_baseline_cal["Ch0"].iloc[0:10]}, title="Baselined Sensitivity Fit", save=False)
    

progress.value += 5

In [166]:
import pickle as pkl

pkl.dump(Tflip_cal, open("Tflip_cal.pkl", "wb"))

## Extract Information from the Calibration Waveforms  
- Pull out the amplitude, charge, and timing from the data. Uses the `analyzeWaveforms()` function.
- Find the average charge, amplitudem and timing for each channel (i.e, each detector in the calibration.)

In [ ]:
# Define a parallelized function
results = Parallel(n_jobs=-1)(
    delayed(analyzeWaveforms)(Tflip_cal[ch], j_A=60, j_B=60) for ch in ID
)

# Collect the results
data_cal = {ch: result[0] for ch, result in zip(ID, results)}
J = {ch: result[1] for ch, result in zip(ID, results)}  

# # Calculate the mean value for each channel
# meanData_cal = {ch: data_cal[ch].mean() for ch in CH}
# medData_cal = {ch: data_cal[ch].median() for ch in CH}
# # displayWaveforms(Tflip_cal,  vlines=J, title="Processed Sr-90 Waveforms No Average")

# # Convert to a dataframe
# meanData_cal = pd.DataFrame(meanData_cal).T
# medData_cal = pd.DataFrame(medData_cal).T
# pd.concat([meanData_cal, medData_cal], axis=1)

# # Charge to Energy scalar:
# scale_factor = [2.2/data_cal[ch]["charge"].max() for ch in CH]

scale_factor = 1

# Display baseline reduced waveforms
XY = {ID[0]: Tflip_cal[ID[0]].iloc[0:1000]}

displayWaveforms(XY, title=ID[0]+" Flipped Sensitivity", save=True, vlines=J)

In [ ]:
Reduced_cal = {i: Tflip_cal[i].iloc[0:1000] for i in ID}

displayWaveforms(Reduced_cal, title="Flipped Sensitivity", save=True, vlines=J, multi=True)

## Construct Energy Distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from scipy.optimize import curve_fit


# Assuming 'hist' is the histogram data and 'bin_centers' are the corresponding x values
def integrate_spectrum(bin_centers, hist, fit_function=None, popt=None):
    """
    Integrate the spectrum either directly from the histogram or using the fit function.
    
    Parameters:
    - bin_centers: The bin center values (x-axis)
    - hist: The histogram values (y-axis)
    - fit_function: The fit function (e.g., Gaussian or double Gaussian) [Optional]
    - popt: The parameters of the fit function [Optional]
    
    Returns:
    - integral: The area under the curve
    """
    if fit_function and popt is not None:
        # Use the fit function to calculate the curve
        y_fit = fit_function(bin_centers, *popt)
        integral = simpson(y_fit, x=bin_centers)
    else:
        # Integrate the raw histogram
        integral = simpson(hist, x=bin_centers)
    
    return integral


def gaussian(x, amp, mu, sigma):
    return amp * np.exp(-0.5 * ((x - mu) / sigma)**2)


# Set up the subplots
nrows = 3
ncols = 3
fig, axes = plt.subplots(nrows, ncols, figsize=(8, 5), sharey=True)
if nrows > 1 and ncols > 1:
    axes = axes.ravel()
else:
    axes = [axes]

MPV_dict = {ch: None for ch in ID}  # Dictionary to store MPVs
bins = 2000  # Number of bins
i = 0

for ax in axes:
    # Create a histogram
    hist, bin_edges = np.histogram(data_cal[ID[i]]["charge"]*scale_factor, bins=bins, density=True)
        
    # Calculate bin centers
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Crop out negative values
    hist = hist[(bin_centers > 0) & (bin_centers < 800)]
    bin_centers = bin_centers[(bin_centers > 0) & (bin_centers < 800)]
        
    # Plot the histogram
    ax.plot(bin_centers, hist, label="Sensitivity")
    ax.set_title(f"Run {ID[i]}")
    ax.set_ylabel("Counts")
    
    if i > 5:
        ax.set_xlabel("Charge (arb)")
    
    try:
        # Initial guesses for the two peaks (adjust based on your data):
        p0 = [
            np.max(hist[:]),                      # Amplitude of peak 1
            bin_centers[:][np.argmax(hist[:])],     # Mean of peak 1
            200,                             # Standard deviation of peak 1
        ]
        
        # Fit the spectrum (double Gaussian example)
        popt, pcov = curve_fit(gaussian, bin_centers, hist, p0=p0)  # Provide initial guesses
        
        # Integrate the fitted curve
        integral_fit = integrate_spectrum(bin_centers, hist, fit_function=gaussian, popt=popt)
        
        # Integrate the raw histogram
        integral_raw = integrate_spectrum(bin_centers, hist)
        
        print(f"Channel {ID[i]}: Total Integral (Fit): {integral_fit}, Raw Histogram: {integral_raw}")
        
        # Extract the means (mu1, mu2) as the most probable values for each peak
        mpv1 = popt[1]
        mu1_error = np.sqrt(pcov[1, 1])
        
        print(popt)
        
        MPV_dict[ID[i]] = {"MPV1": mpv1, "MPV1_error": mu1_error, "Integral": integral_fit}
        
        print(f"Channel {ID[i]}: MPV1 = {mpv1} ± {mu1_error}")
        
        # Plot the fitted curve and histogram
        ax.plot(bin_centers, gaussian(bin_centers, *popt), 'r--', label="Fit")
        
    except Exception as e:
        print(f"Unable to fit Channel {ID[i]}: {e}")
    
    
    # Print average energy
    # Calculate bin centers
    # bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Calculate the average x-value
    average_x = np.sum(bin_centers * hist) / np.sum(hist)
    
    # ax.vlines(mpv1, hist.min(), hist.max(), color="black", linestyles="--", label="<E>={:.2f} MeV".format(average_x))
    # ax.set_yscale('log')
    # ax.legend()
    print(average_x)
    
    i += 1

# Set the overall title for the figure
# fig.suptitle("Muon Spectrum", fontsize=16)

# Tight layout for spacing
plt.tight_layout()

plt.savefig(f"Figures/Charge.png", dpi=300)
# # Scale MPVs based on calibration
# mpv_scaled = [(MPV_dict[CH[i]][0] / CalScale[i], MPV_dict[CH[i]][1] / CalScale[i]) for i in range(4)]
# print(mpv_scaled)

print(MPV_dict)

In [258]:
MPVs = [MPV_dict[ID[i]]["Integral"] for i in range(len(ID))]
Locs = [RunLocMap[ID[i]] for i in range(len(ID))]

plt.figure(figsize=(8, 5))
plt.scatter(Locs, MPVs)
plt.savefig("MPVs.png", dpi=300)